## 数据预处理与特征工程

In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from collections import Counter
import pickle
import time

### 1. train数据处理  

In [ ]:
dpath = "./data/"
train = pd.read_csv(dpath+"train.csv")
train.head()

把特征取值较少的类别进行了合并，填补缺失值

In [ ]:
def train_cleaner(input_file,output_file): 
    """
    function:
        clean train data and write clean train data into output_file
    params:
        input_file: input file path
        output_file: output file path
    """
    fin = open(dpath+input_file,"r+")
    fout = open(dpath+output_file,"w+")
    ocolnames = ["msno","song_id","source_system_tab","source_screen_name","source_type","target"]
    fout.write(",".join(ocolnames) + "\n")  
    start = 0
    for line in fin:
        if start == 0:
            start+=1
            continue
        cols = line.strip().split(",")
        if cols[2] in ["","notification","settings"]:
            cols[2] = "other system"
        if cols[3] in ["Discover New","Search Trends","Search Home",\
                       "My library_Search","Self profile more","Concert","Payment"]:
            cols[3] = "other screen"
        if cols[3] == "":
            cols[3] = "screen nan"
        if cols[4] in ["","topic-article-playlist","artist","my-daily-playlist"]:
            cols[4] = "other type"
        fout.write(",".join(cols)+"\n")
    fin.close()
    fout.close()

类别特征LabelEncoder

bug1: '<' not supported between instances of 'str' and 'float'  
解决：.astype(str)

In [ ]:
def train_encode(input_file):
    train_clean = pd.read_csv(dpath+input_file)
    le = preprocessing.LabelEncoder()
    colnames = ["source_system_tab","source_screen_name","source_type"]
    for colname in colnames:
        if colname == "source_system_tab":
            cols_2 = le.fit_transform(train_clean[colname].astype(str))
        if colname == "source_screen_name":
            cols_3 = le.fit_transform(train_clean[colname].astype(str))
        if colname == "source_type":
            cols_4 = le.fit_transform(train_clean[colname].astype(str))
    return cols_2,cols_3,cols_4

In [ ]:
train_encode("train_clean.csv")

把处理好的train数据写入文件

In [ ]:
def generate_train_data(input_file,output_file):
    ocolnames = ["msno","song_id","source_system_tab","source_screen_name","source_type","target"]
    fin = open(dpath+input_file,"r+")
    fout = open(dpath+output_file,"w+")
    fout.write(",".join(ocolnames)+"\n")
    i = 0
    start = 0
    cols_2,cols_3,cols_4 = train_encode(input_file)
    for line in fin:
        if start == 0:
            start+=1
            continue
        cols = line.strip().split(",")
        cols[2] = str(cols_2[i])
        cols[3] = str(cols_3[i])
        cols[4] = str(cols_4[i])
        fout.write(",".join(cols)+"\n")
        i += 1
        #文件遍历指针在第7377418行，舍去最后空行
        if i == 7377418:
            break
    fin.close()
    fout.close()    

In [ ]:
train_cleaner("train.csv","train_clean.csv")
generate_train_data("train_clean.csv","train_data.csv")

In [ ]:
train_data = pd.read_csv(dpath+"train_data.csv")
train_data.head()

### 2.test数据处理

In [ ]:
def test_cleaner(input_file,output_file): 
    """
    function:
        clean test data and write test data into output_file
    params:
        input_file: input file path
        output_file: output file path
    """
    fin = open(dpath+input_file,"r+")
    fout = open(dpath+output_file,"w+")
    ocolnames = ["id","msno","song_id","source_system_tab","source_screen_name","source_type"]
    fout.write(",".join(ocolnames) + "\n")  
    start = 0
    for line in fin:
        if start == 0:
            start+=1
            continue
        cols = line.strip().split(",")
        if cols[3] in ["","notification","settings"]:
            cols[3] = "other system"
        if cols[4] in ["Discover New","Search Trends","Search Home",\
                       "My library_Search","Self profile more","Concert","Payment"]:
            cols[4] = "other screen"
        if cols[4] == "":
            cols[4] = "screen nan"
        if cols[5] in ["","topic-article-playlist","artist","my-daily-playlist"]:
            cols[5] = "other type"
        fout.write(",".join(cols)+"\n")
    fin.close()
    fout.close()

In [ ]:
def test_encode(input_file):
    test_clean = pd.read_csv(dpath+input_file)
    le = preprocessing.LabelEncoder()
    colnames = ["source_system_tab","source_screen_name","source_type"]
    for colname in colnames:
        if colname == "source_system_tab":
            cols_2 = le.fit_transform(test_clean[colname].astype(str))
        if colname == "source_screen_name":
            cols_3 = le.fit_transform(test_clean[colname].astype(str))
        if colname == "source_type":
            cols_4 = le.fit_transform(test_clean[colname].astype(str))
    return cols_2,cols_3,cols_4

In [ ]:
def generate_test_data(input_file,output_file):
    ocolnames = ["id","msno","song_id","source_system_tab","source_screen_name","source_type"]
    fin = open(dpath+input_file,"r+")
    fout = open(dpath+output_file,"w+")
    fout.write(",".join(ocolnames)+"\n")
    i = 0
    start = 0
    cols_3,cols_4,cols_5 = test_encode(input_file)
    for line in fin:
        if start == 0:
            start+=1
            continue
        cols = line.strip().split(",")
        cols[3] = str(cols_3[i])
        cols[4] = str(cols_4[i])
        cols[5] = str(cols_5[i])
        fout.write(",".join(cols)+"\n")
        i += 1
        #文件遍历指针在第2556790行，舍去最后空行，不然会报错
        if i == 2556790:
            break
    fin.close()
    fout.close()    

In [ ]:
test_cleaner("test.csv","test_clean.csv")
generate_test_data("test_clean.csv","test_data.csv")

In [ ]:
test_data = pd.read_csv(dpath+"test_data.csv")
test_data.head()

### 3.songs数据处理

In [ ]:
songs = pd.read_csv(dpath+"songs.csv")
songs.head()

In [ ]:
songs.info()

清洗songs，增加mult_genre特征

In [ ]:
def songs_cleaner(input_file,output_file): 
    """
    function:
        clean songs data and write songs data into output_file
    params:
        input_file: input file path
        output_file: output file path
    """
    #注意这里的encoding='UTF-8'，不然打开文件报错，为什么？
    fin = open(dpath+input_file,"r+",encoding='UTF-8')
    fout = open(dpath+output_file,"w+")
    ocolnames = ["song_id","song_length","genre_ids","language","mult_genre"]
    fout.write(",".join(ocolnames) + "\n")
    start = 0
    for line in fin:
        mult_genre = "0"
        if start == 0:
            start+=1
            continue
        cols = line.strip().split(",")
        if cols[2] == "":
            cols[2] = "1234"
        if "|" in cols[2]:
            cols[2] = cols[2][0]
            mult_genre = "1"
        #输出列，丢掉cols[3],cols[4],cols[5]
        outcols = [cols[0],cols[1],cols[2],cols[6],mult_genre]
        fout.write(",".join(outcols)+"\n")
    fin.close()
    fout.close()

genre_ids归类，少数类别划分为small_1,small_2,small_3

In [ ]:
def merge_category(input_file,output_file):
    data = pd.read_csv(dpath+input_file)
    small_list1 = []
    small_list2 = []
    small_list3 = []
    feature_count = data["genre_ids"].value_counts()
    for i in range(len(feature_count.values)):
        if feature_count.values[i]<10000:
            small_list1.append(feature_count.index[i])
        if feature_count.values[i]>=10000 and feature_count.values[i]<15000:
            small_list2.append(feature_count.index[i])
        if feature_count.values[i]>=15000 and feature_count.values[i]<20000:
            small_list3.append(feature_count.index[i])
    #else不能省略，注意这里要赋值给data["genre_ids"],不然未修改原df
    data["genre_ids"] = data.genre_ids.apply(lambda x:"small_1" if x in small_list1 else x)
    data["genre_ids"] = data.genre_ids.apply(lambda x:"small_2" if x in small_list2 else x)
    data["genre_ids"] = data.genre_ids.apply(lambda x:"small_3" if x in small_list3 else x)
    data.to_csv(dpath+output_file,index=False)

songs_clean类别型数据LabelEncoder，连续型做StandardScaler

In [ ]:
def songs_encode(input_file):
    songs_clean = pd.read_csv(dpath+input_file)
    #song_lenth标准化
    ss = StandardScaler()
    #不接受1维数组，reshape成二维数组，注意后面写入文件时如何取值
    song_length = np.array(songs_clean["song_length"]).reshape(-1,1)
    song_length = ss.fit_transform(song_length)
    song_length = np.around(song_length,decimals=5)
    #genre_ids和language编码
    le = preprocessing.LabelEncoder()
    colnames = ["genre_ids","language"]
    for colname in colnames:
        if colname == "genre_ids":
            genre_ids = le.fit_transform(songs_clean[colname].astype(str))
        if colname == "language":
            language = le.fit_transform(songs_clean[colname].astype(str))
    return song_length,genre_ids,language

把songs清洗好的特征列写入文件

In [ ]:
def generate_songs_data(input_file,output_file):
    ocolnames = ["song_id","song_length","genre_ids","language","mult_genre"]
    fin = open(dpath+input_file,"r+")
    fout = open(dpath+output_file,"w+")
    fout.write(",".join(ocolnames)+"\n")
    i = 0
    start = 0
    song_length,genre_ids,language = songs_encode("songs_clean.csv")
    for line in fin:
        if start == 0:
            start+=1
            continue
        cols = line.strip().split(",")
        #cols_1是(-1,1)二维数组，cols_1[i][0]取出值
        cols[1] = str(song_length[i][0])
        cols[2] = str(genre_ids[i])
        #合并少数language,[2,7,9,6]
        if language[i] in [7,9,6]:
            language[i] = 2
        cols[3] = str(language[i])
        fout.write(",".join(cols)+"\n")
        i += 1
        #文件遍历指针在第2296320行，舍去最后空行
        if i == 2296320:
            break
    fin.close()
    fout.close()    

In [ ]:
songs_cleaner("songs.csv","songs_clean.csv")
merge_category("songs_clean.csv","songs_clean.csv")
generate_songs_data("songs_clean.csv","songs_data.csv")

In [ ]:
songs_data = pd.read_csv(dpath+"songs_data.csv")
songs_data.head()

In [ ]:
songs_data.shape

In [ ]:
len(songs_data["genre_ids"].unique())

### 4.members数据处理

In [ ]:
members = pd.read_csv(dpath + "members.csv")
members.head()

清洗特征

In [ ]:
def members_encode(members_file):
    memmbers = pd.read_csv(dpath+members_file)
    bd = members["bd"].apply(lambda x : 0 if x<0 or x>100 else x)
    bd = pd.cut(bd.values,bins=[-1,6,12,18,22,25,30,35,40,50,60,100],labels=False)
    #pd.cut输入数据类型要求
    registration_init_time = pd.cut(members["registration_init_time"].values,\
                        bins=range(20040000,20190000,10000),labels=False)
    expiration_date = members["expiration_date"].apply(lambda\
                  x : 20170930 if x==19700101 else x)
    expiration_date = pd.cut(expiration_date.values,\
                        bins=range(20040000,20220000,10000),labels=False)
    return bd, registration_init_time, expiration_date

把清洗好的特征写入文件

In [ ]:
def generate_members_data(input_file,output_file):
    ocolnames = ["msno","city","bd","gender","registered_via",\
                 "registration_init_time","expiration_date"]
    fin = open(dpath+input_file,"r+")
    fout = open(dpath+output_file,"w+")
    fout.write(",".join(ocolnames)+"\n")
    bd,registration_init_time,expiration_date = members_encode(input_file)
    i = 0
    start = 0
    for line in fin:
        if start == 0:
            start+=1
            continue
        cols = line.strip().split(",")
        msno = cols[0]
        city = cols[1]
        registered_via = cols[4]
        if cols[3] == "female":
            gender = "0"
        elif cols[3] == "male":
            gender = "1"
        else:
            gender = "2"
        if expiration_date[i] == "":
            expiration_date[i] = 16
        outcols = [msno,city,str(bd[i]),gender,registered_via,\
                  str(registration_init_time[i]),str(expiration_date[i])]
        fout.write(",".join(outcols)+"\n")
        i += 1
        #文件遍历指针在第34403行，舍去最后空行
        if i == 34403:
            break
    fin.close()
    fout.close()    

In [ ]:
generate_members_data("members.csv","members_data.csv")

In [ ]:
members_data = pd.read_csv(dpath+"members_data.csv")
members_data.head()

### 5.生成最终训练和测试文件

把songs_data,members_data转化为字典存储  
"song_id": ["song_length","genre_ids","language","mult_genre"]  
"msno": ["city","bd","gender","registered_via","registration_init_time","expiration_date"]

In [ ]:
def generate_info_dict(input_file,output_file):
    fin = open(dpath+input_file,"r+")
    info_dict = dict()
    start = 0
    for line in fin:
        if start == 0:
            start+=1
            continue
        cols = line.strip().split(",")
        if cols[0] not in info_dict:
            info_dict[cols[0]] = []
        info_dict[cols[0]] = cols[1:]
    pickle.dump(info_dict,open(dpath+output_file,"wb"))

合并train，songs，memmbers数据，生成最终训练文件  
train数据集通过歌曲id和用户id映射到songs和members特征

In [4]:
dpath = "./data/"

In [5]:
def generate_train_merge(input_file,output_file):
    songs_dict = pickle.load(open(dpath+"songs_dict.pkl","rb"))
    members_dict = pickle.load(open(dpath+"members_dict.pkl","rb"))
    fin = open(dpath+input_file,"r+")
    fout = open(dpath+output_file,"w+")
    #注意ocolnames顺序要和后面outcols对应
    train_cols = ["msno","song_id","source_system_tab","source_screen_name","source_type","target"]
    songs_cols = ["song_length","genre_ids","language","mult_genre"]
    members_cols = ["city","bd","gender","registered_via","registration_init_time","expiration_date"]
    ocolnames = train_cols[:5]+songs_cols+members_cols+["target"]
    fout.write(",".join(ocolnames)+"\n")
    outcols = []
    for line in fin:
        cols = line.strip().split(",")
        #只在训练集或测试集出现的members和songs如何处理
        if cols[0] in members_dict and cols[1] in songs_dict:
            #合并3组特征并写入文件，注意cols[5]是个str，需要转换成list
            outcols = cols[:5]+songs_dict[cols[1]]+members_dict[cols[0]]+[cols[5]]
        else:
            continue
        fout.write(",".join(outcols)+"\n")
    fout.close()  

In [ ]:
generate_info_dict("songs_data.csv","songs_dict.pkl")
generate_info_dict("members_data.csv","members_dict.pkl")

In [6]:
generate_train_merge("train_data.csv","train_merge.csv")

In [9]:
# fin = open(dpath+"train_data.csv","r+")
# songs_dict = pickle.load(open(dpath+"songs_dict.pkl","rb"))
# count = 0
# for line in fin:
#     cols = line.strip().split(",")
#     if cols[1] in songs_dict:
#         count+=1

In [10]:
count

7377403

In [11]:
train_merge = pd.read_csv(dpath+"train_merge.csv")
train_merge.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,song_length,genre_ids,language,mult_genre,city,bd,gender,registered_via,registration_init_time,expiration_date,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,1,5,4,-0.25183,10,8,0,1,0,2,7,8,13,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,3,6,3,0.23361,4,8,0,13,4,0,9,7,13,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,3,6,3,-0.13422,4,8,0,13,4,0,9,7,13,1
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,3,6,3,0.05294,17,0,0,13,4,0,9,7,13,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,1,5,4,-0.36784,1,8,0,1,0,2,7,8,13,1


In [12]:
train_merge.shape

(7377403, 16)

In [13]:
train_merge.apply(lambda x:sum(x.isnull()))

msno                      0
song_id                   0
source_system_tab         0
source_screen_name        0
source_type               0
song_length               0
genre_ids                 0
language                  0
mult_genre                0
city                      0
bd                        0
gender                    0
registered_via            0
registration_init_time    0
expiration_date           0
target                    0
dtype: int64

合并test，songs，memmbers数据，生成最终测试文件

In [ ]:
def generate_test_final(input_file,output_file):
    songs_dict = pickle.load(open(dpath+"songs_dict.pkl","rb"))
    members_dict = pickle.load(open(dpath+"members_dict.pkl","rb"))
    fin = open(dpath+input_file,"r+")
    fout = open(dpath+output_file,"w+")
    #注意ocolnames顺序要和后面outcols对应
    #test多了"id",没有"target"
    test_cols = ["id","msno","song_id","source_system_tab","source_screen_name","source_type"]
    songs_cols = ["song_length","genre_ids","language","mult_genre"]
    members_cols = ["city","bd","gender","registered_via","registration_init_time","expiration_date"]
    ocolnames = test_cols+songs_cols+members_cols
    fout.write(",".join(ocolnames)+"\n")
    outcols = []
    for line in fin:
        cols = line.strip().split(",")
        #注意这里cols索引，test多了第一列id，过滤掉只在test中出现的用户，这些用户该如何处理？
        if cols[1] in members_dict and cols[2] in songs_dict:
            #合并3组特征并写入文件，注意cols[5]是个str，需要转换成list
            outcols = cols+songs_dict[cols[2]]+members_dict[cols[1]]
        fout.write(",".join(outcols)+"\n")
    fout.close()  

In [ ]:
generate_test_final("test_data.csv","test_final.csv")

In [ ]:
test_final = pd.read_csv(dpath+"test_final.csv")
test_final.head()

In [ ]:
test_final.shape